# Board games scraping

In [ ]:
import requests as re
from bs4 import BeautifulSoup
import pandas as pd
import json
from datetime import datetime
import time
from tqdm.notebook import tqdm_notebook

In [ ]:
def bgg_top(n_pages=20, start_page=1):
    '''
    On this script you can get list of top board games from boardgamegeek.com
    -----
    Paramrters
    -----
    n_pages: how many pages you searh, default - 20, because without sighning in on boardgamegeek.com you can load only first 20 pages
    start_page: from this page we start scrapping, default - 1.
    '''
    url_main = 'https://boardgamegeek.com'
    url_searh_boardgames = url_main + '/browse/boardgame'
    url_page = '/page/'
    # n_pages = 20
    game_top_list = pd.DataFrame()
    for i in range(n_pages):
        r = re.get(url_searh_boardgames + url_page + str(start_page+i))
        soup = BeautifulSoup(r.text)
        rows = soup.find_all('tr', id='row_')
        for j, row in enumerate(rows):
            rank = row.find('td', class_='collection_rank').find('a').get('name')
            gameinfo = row.find('div', id='results_objectname'+str(j+1))
            title = gameinfo.find('a').text
            try:
                year = gameinfo.find('span').text
            except:
                year = ''
            try:
                description = row.find('p', class_='smallefont dull').text.strip()
            except:
                description = ''
            link = url_main + gameinfo.find('a').get('href')
            game_id = str(gameinfo.find('a').get('href'))[len('/boardgame/'):]
            game_id = game_id[:game_id.find('/')]
            ratings = row.find_all('td', class_='collection_bggrating')
            rat_list = []
            for i in ratings:
                rat_list.append(i.text.strip())
            geek_rating = rat_list[0]
            avg_rating = rat_list[1]
            num_votes = rat_list[2]

            game = {'rank': rank, 'title': title, 'game_id':game_id, 'description':description, 'year':year, 'link': link, 
                    'geek_rating':geek_rating, 'avg_rating':avg_rating, 'num_votes':num_votes}
            game_top_list = pd.concat([game_top_list, pd.DataFrame([game])])

    return game_top_list

In [ ]:
def get_bgg_categories():
    category_list = pd.DataFrame()
    url_main = 'https://boardgamegeek.com'
    url_cat = 'https://boardgamegeek.com/browse/boardgamecategory'
    r = re.get(url_cat)
    soup = BeautifulSoup(r.text)
    table = soup.find('table', class_='forum_table')
    rows = table.find_all('a')
    for row in rows:
        link = url_main + row.get('href')
        category = row.text
        cat = {'category':category, 'link':link}
        category_list = pd.concat([category_list, pd.DataFrame([cat])])
    return category_list